In [20]:
import random
from collections import Counter

class Tile:
    def __init__(self, color):
        self.color = color

    def get_color(self):
        return self.color

class TileBag:
    def __init__(self):
        self.tiles = []
        self.init_tiles()
        random.seed()

    def init_tiles(self):
        colors = ["Black", "Red", "Blue", "Yellow", "Green"]
        tiles_per_color = 20

        for color in colors:
            for _ in range(tiles_per_color):
                self.tiles.append(Tile(color))

    def draw_random_tile(self):
        if not self.tiles:
            raise Exception("No tiles left in the bag")

        random_tile = random.choice(self.tiles)
        self.tiles.remove(random_tile)
        return random_tile

    def print_contents(self):
        print_tile_counts(self.tiles, "Tile Bag")

class CentralMarket:
    def __init__(self):
        self.tiles = []
        self.center_taken = False

    def add_tile(self, tile):
        self.tiles.append(tile)

    def take_tiles_of_color(self, color):
        taken_tiles = [tile for tile in self.tiles if tile.get_color() == color]
        self.tiles = [tile for tile in self.tiles if tile.get_color() != color]
        if not self.center_taken:
            taken_tiles.append(Tile("First Player"))
            self.center_taken = True
        return taken_tiles

    def is_empty(self):
        return not self.tiles
    
    def get_contents(self):
        return Counter([tile.get_color() for tile in self.tiles])

    def print_contents(self):
        print_tile_counts(self.tiles, "Central Market")

class FactoryDisplay:
    def __init__(self, tile_bag, display_id=0):
        self.tiles = []
        self.refill(tile_bag)
        self.display_id = display_id

    def refill(self, tile_bag):
        self.tiles = [tile_bag.draw_random_tile() for _ in range(4)]

    def take_tiles_of_color(self, color):
        taken_tiles = [tile for tile in self.tiles if tile.get_color() == color]
        discarded_tiles = [tile for tile in self.tiles if tile.get_color() != color]
        self.tiles = []
        return taken_tiles, discarded_tiles

    def is_empty(self):
        return not self.tiles

    def get_contents(self):
        return Counter([tile.get_color() for tile in self.tiles])
    
    def print_contents(self):
        if self.is_empty():
            print(f"Factory Display {self.display_id} is empty")
        else:
            print_tile_counts(self.tiles, f"Factory Display {self.display_id}")

class Factory:
    def __init__(self, num_factory_displays):
        self.tile_bag = TileBag()
        self.central_market = CentralMarket()
        self.factory_displays = [FactoryDisplay(self.tile_bag,id) for id in range(num_factory_displays)]

    def refill_displays(self):
        if [display.is_empty() for display in self.factory_displays]:
            [display.refill(self.tile_bag) for display in self.factory_displays]

    def take_display_tiles(self, display_id, color):
        # Check the display has the color
        display = self.factory_displays[display_id]
        if not any(tile.get_color() == color for tile in display.tiles):
            return False
        else:
            taken_tiles, discarded_tiles = display.take_tiles_of_color(color)
            # Rest goes to central market
            for tile in discarded_tiles:
                self.central_market.add_tile(tile)
            return taken_tiles
        
    def take_center_tiles(self, color):
        taken_tiles = self.central_market.take_tiles_of_color(color)
        return taken_tiles
    
    def get_contents(self, display_id=None):
        """Get the contents of the factory displays and central market
        display_id None, returns the central market contents"""
        if display_id is None:
            return Counter([tile.get_color() for tile in self.central_market.tiles])
        else:
            return Counter([tile.get_color() for tile in self.factory_displays[display_id].tiles])


    
    def print_contents(self):
        self.tile_bag.print_contents()
        self.central_market.print_contents()
        for i, display in enumerate(self.factory_displays):
            display.print_contents()


def print_tile_counts(tiles, container_name):
    color_counts = Counter([tile.get_color() for tile in tiles])
    # Sort by color
    color_counts = {k: v for k, v in sorted(color_counts.items(), key=lambda item: item[0])}
    print(f"{container_name} contents: {color_counts}")



In [42]:
num_factory_displays = 5
factory = Factory(num_factory_displays)

factory.print_contents()

Tile Bag contents: {'Black': 14, 'Blue': 16, 'Green': 17, 'Red': 17, 'Yellow': 16}
Central Market contents: {}
Factory Display 0 contents: {'Black': 2, 'Blue': 1, 'Red': 1}
Factory Display 1 contents: {'Black': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}
Factory Display 2 contents: {'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}
Factory Display 3 contents: {'Black': 2, 'Blue': 2}
Factory Display 4 contents: {'Black': 1, 'Green': 1, 'Yellow': 2}


In [ ]:
factory.print_contents()
factory.refill_displays()
factory.print_contents()


In [43]:
# Refil and empty
for round in range(5):
    # Take all the tiles
    factory.print_contents()
    for id in range(num_factory_displays):
        if factory.get_contents(id):
            top_color = factory.get_contents(id).most_common(1)[0][0]
        my_tiles = factory.take_display_tiles(id, top_color)
    factory.print_contents()
    colors = ["Black", "Red", "Blue", "Yellow", "Green"]
    for color in colors:
        my_tiles = factory.take_center_tiles(color)

    factory.print_contents()
    factory.refill_displays()
    factory.print_contents()

    


Tile Bag contents: {'Black': 14, 'Blue': 16, 'Green': 17, 'Red': 17, 'Yellow': 16}
Central Market contents: {'Blue': 1, 'Red': 1}
Factory Display 0 is empty
Factory Display 1 contents: {'Black': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}
Factory Display 2 contents: {'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}
Factory Display 3 contents: {'Black': 2, 'Blue': 2}
Factory Display 4 contents: {'Black': 1, 'Green': 1, 'Yellow': 2}
Tile Bag contents: {'Black': 14, 'Blue': 16, 'Green': 17, 'Red': 17, 'Yellow': 16}
Central Market contents: {'Blue': 1, 'Green': 1, 'Red': 2, 'Yellow': 1}
Factory Display 0 is empty
Factory Display 1 is empty
Factory Display 2 contents: {'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}
Factory Display 3 contents: {'Black': 2, 'Blue': 2}
Factory Display 4 contents: {'Black': 1, 'Green': 1, 'Yellow': 2}
Tile Bag contents: {'Black': 14, 'Blue': 16, 'Green': 17, 'Red': 17, 'Yellow': 16}
Central Market contents: {'Blue': 1, 'Green': 2, 'Red': 3, 'Yellow': 2}
Factory Display 0

Exception: No tiles left in the bag